In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
torch.set_printoptions(edgeitems=2)
torch.manual_seed(123)

#### when looking to make a neural network we need to decide upon the activation function first.
#### we have many options available to us

In [2]:
t_c = [0.5,  14.0, 15.0, 28.0, 11.0,  8.0,  3.0, -4.0,  6.0, 13.0, 21.0]
t_u = [35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4]
t_c = torch.tensor(t_c).unsqueeze(1) # this unsqueeze is being done to parallelize the input
t_u = torch.tensor(t_u).unsqueeze(1) # <1>

t_u.shape

torch.Size([11, 1])

In [3]:
n_samples = t_u.shape[0]
n_val = int(0.2 * n_samples)

shuffled_indices = torch.randperm(n_samples)

train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]

train_indices, val_indices

(tensor([ 3, 10,  5,  2,  7,  4,  0,  8,  1]), tensor([9, 6]))

In [4]:
t_u_train = t_u[train_indices]
t_c_train = t_c[train_indices]

t_u_val = t_u[val_indices]
t_c_val = t_c[val_indices]

t_un_train = 0.1 * t_u_train
t_un_val = 0.1 * t_u_val

In [5]:
#our basic neuron is built from the torch.nn submodule
#we solve our temperature conversion using a linear neuron unit
import torch.nn as nn

linear_model = nn.Linear(1, 1) # <1>
linear_model(t_un_val)


tensor([[-1.8575],
        [-0.8784]], grad_fn=<AddmmBackward>)

Every submodule of the nn.Module has a function \__call__ which allows us to use it as if it were a regular function. 
The diff between \__call__ and forward is that the latter just does a simple forward pass, but \__call__ does other relavant stuff before and after calling forward itself

In [6]:
#__call__ is simply done as below
# y = model(x)
# using forward directly as shown below is not the correct way of computing
# y = model.forward(x)

In [7]:
#Returning to the linear model
import torch.nn as nn
linear_model = nn.Linear(1, 1) #1 and 1 are the sizes of input and output tensors
linear_model(t_un_val)

tensor([[-5.7330],
        [-3.4839]], grad_fn=<AddmmBackward>)

In [8]:
#we can check weights and biases as follows
linear_model.weight, linear_model.bias

(Parameter containing:
 tensor([[-0.8487]], requires_grad=True),
 Parameter containing:
 tensor([-0.6067], requires_grad=True))

In [9]:
#calling model with sample input
x = torch.ones(1)
linear_model(x)

tensor([-1.4555], grad_fn=<AddBackward0>)

Usually pytorch expects inputs to be in batches, with the zeroth input being the number of batches. 

We can batch our inputs as follows

In [10]:
x = torch.ones(10, 1)
linear_model(x)


tensor([[-1.4555],
        [-1.4555],
        [-1.4555],
        [-1.4555],
        [-1.4555],
        [-1.4555],
        [-1.4555],
        [-1.4555],
        [-1.4555],
        [-1.4555]], grad_fn=<AddmmBackward>)

In [11]:
# changing our model to use an optimizer
linear_model = nn.Linear(1, 1)
optimizer = optim.SGD(linear_model.parameters(), lr=1e-2)